# 12.REINFORCE算法
> 1992 年，Ronald J. Williams 的论文[《Simple statistical gradient-following algorithms for connectionist reinforcement learning》](https://link.springer.com/article/10.1007/BF00992696)首次系统地提出了 **基于蒙特卡洛** 回报的策略梯度更新方法：**REINFORCE**，它是现代策略梯度方法的起点，是第一个系统性提出并形式化了策略梯度更新规则的算法

**REINFORCE** 算法采用 **蒙特卡洛方法** 来估计 $Q^{\pi_{\theta}}(s,a)$ ，对于一个有限步数的环境来说，REINFORCE 算法中的策略梯度为：

## REINFORCE 代码实践：

导入相关库：

In [ ]:
# 基本库
import numpy as np
from tqdm import tqdm
"""在绘图时引入,防止绘图时失效
import plotly.graph_objects as go
import pandas as pd
"""
# 神经网络
import torch
import torch.nn.functional as F
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

定义策略网络：

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)  # 输出通过 softmax 激活函数转化为一个概率分布，表示每个动作的选择概率

定义 REINFORCE 算法：

In [ ]:
class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma, device):
        self.policy_net = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=learning_rate)  # 使用Adam优化器
        self.gamma = gamma  # 折扣因子
        self.device = device

    def take_action(self, state):  # 根据动作概率分布随机采样
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        probs = self.policy_net(state)  # 输出一个概率分布
        action_dist = torch.distributions.Categorical(probs)  # 创建一个 Categorical 分布，方便从中采样
        action = action_dist.sample()  # 从概率分布中随机采样一个动作的索引
        return action.item()

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']
        G = 0
        self.optimizer.zero_grad()
        for i in reversed(range(len(reward_list))):  # 从最后一步算起
            reward = reward_list[i]
            state = torch.tensor([state_list[i]],
                                 dtype=torch.float).to(self.device)
            action = torch.tensor([action_list[i]]).view(-1, 1).to(self.device)
            log_prob = torch.log(self.policy_net(state).gather(1, action))
            G = self.gamma * G + reward
            loss = -log_prob * G  # 每一步的损失函数
            loss.backward()  # 反向传播计算梯度
        self.optimizer.step()  # 梯度下降